# *Bounded Model Checking*

Nesta aula vamos estudar *model checking*, uma das técnicas mais utilizadas para verificação automática de propriedades de sistemas dinâmicos (sistemas caracterizados por um estado que evolui ao longo do tempo). O *model checking* tem por objectivo verificar automaticamente se uma propriedade (tipicamente especificada numa lógica temporal) é válida num modelo do sistema (tipicamente formalizado como um sistema de transição). Mais concretamente vamos estudar uma técnica de *bounded model checking*, onde o objectivo é verificar se uma propriedade é válida considerando apenas um dado número máximo de estados da execução do sistema.

Vamos usar como SMT solver o popular solver Z3, através da biblioteca Python  Z3Py.

## Brevíssima introdução à utilização do Z3 em Python

Começamos por instalar o Z3.

In [ ]:
!pip install z3-solver

A documentação do Z3py pode ser encontrada em https://ericpony.github.io/z3py-tutorial/guide-examples.htm.

A utilização desta biblioteca segue normalmente os seguintes passos:

1. Importar a biblioteca do Z3 usando o comando `from z3 import *`
1. Criar uma instância do solver com a função `Solver()`.
1. Adicionar as variáveis do problema. As funções `Int()`, `Real()`, `Bool()` criam uma variável no Z3 do tipo correspondente. Estas funções recebem o nome da variável como parâmetro.
1. Adicionar as restrições do problema usando o método `add`. A restrição é definida com a sintaxe normal Python para expressões aritméticas e comparações.  As funções `And`, `Or`, `Not` e `Implies` são usadas como os operadores lógicos.

1. Invocar o solver com o método `check`. Este método pode devolver um dos seguintes resultados:
  - `sat`, quando o conjunto de restrições é satisfazível, isto é, quando uma solução for encontrada.
  - `unsat`, quando não é possível resolver o problema, isto é, quando o conjunto de restrições é inconsistente.
  - `unknown`, quando o solver não se consegue pronunciar quanto satisfatibilidade do conjunto de restrições.

1. Interpretar os resultados no caso do resultado ser `sat`. O método `model` dá o modelo produzido pelo solver (o modelo associa a cada variável o valor que lhe foi atribuido na solução encontrada).
Podemos consultar o modelo gerado. O método `decls` devolve as variáveis atribuídas no modelo. O método `name` devolve o nome de uma variável atribuída no modelo. O método `eval` permite avaliar uma expressão no modelo. 
Deve ser usado o método `as_long` para converter os valores inteiros do Z3 em inteiros do Python. 



Por exemplo, o programa que se segue tenta encontrar solução para as seguintes restrições:

$$
\left\{
\begin{array}{l}
0 \le x \le 50\\
y \ge x+z \\
z-5y > x \vee 3x+y \le 20
\end{array}
\right.
$$


In [5]:
from z3 import *

s = Solver()
x, y = Ints('x y')
z = Int('z')
s.add(And(0<=x,x<=50))
s.add(y >= x+z)
s.add(Or(z-5*y>x, 3*x+y<=20))

if s.check()==sat :
    m = s.model()
    print(m)
    for d in m.decls():
        print("%s = %d" % (d.name(), m[d].as_long()))
else: 
    print("Não tem solução.")

[x = 50, z = -175, y = -125]
x = 50
z = -175
y = -125


## *First-order Transition Systems* (FOTS)

FOTS são modelos de sistemas dinâmicos que são determinados por um espaço de estados, uma relação de transição de estados e um conjunto de estados iniciais. Nos FOTS o conjunto de estados iniciais é descrito por um predicado unário (*init*) sobre o vector de variáveis associado ao espaço de estados, e a relação de transição é descrita por um predicado binário (*trans*) sobre dois vectores de variáveis que representam o estado antes e depois da transição.

### Modelação de programas com FOTS

Um programa pode ser modelado por um FOTS da seguinte forma:
- O estado é constituído pelas variáveis do programa mais uma variável para o respectivo *program counter*
- Os estados iniciais são caracterizados implicitamente por um predicado sobre as variáveis de estado
- As transições são caracterizadas implicitamente por um predicado sobre pares de estados

Considere, por exemplo o programa seguinte, anotado com uma pré-condição que restringe o valor inicial de $x$:

```Python
{ x >= 3 }
0: while (x>0):
1:    x = x-1
2: stop
```

Neste caso o estado do FOTS respectivo será um par de inteiros, o primeiro contendo o valor do $\mathit{pc}$ (o *program counter* que neste caso pode ser 0, 1 ou 2) e o segundo o valor da variável $x$. O estado inicial é caracterizado pelo seguinte predicado:

$$
\mathit{pc} = 0 \wedge x \ge 3
$$

As transições possíveis no FOTS são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge x > 0 \wedge \mathit{pc}' = 1 \wedge x' = x)\\
\vee\\
(\mathit{pc} = 0 \wedge x \le 0 \wedge \mathit{pc}' = 2 \wedge x' = x) \\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 0 \wedge x' = x - 1) \\
\vee\\
(\mathit{pc} = 2 \wedge \mathit{pc}' = 2 \wedge x' = x)
\end{array}
$$

Note que este predicado é uma disjunção de todas as possíveis transições que podem ocorrer no programa. Cada transição é caracterizada por um predicado onde uma variável do programa denota o seu valor no pré-estado e a mesma variável com apóstrofe denota o seu valor no pós-estado. É usual exigir que cada estado do FOTS tenha pelo menos um sucessor, pelo que o estado final do programa é caracterizado por uma transição para ele próprio (um lacete).

Usando estes predicados podemos usar um SMT solver (nomeadamente o Z3) para, por exemplo, gerar uma possível execução de $k\!-\!1$ passos do programa (em que $k>0$). Para tal precisamos de criar $k$ cópias das variáveis que caracterizam o estado do FOTS e depois impor que a primeira cópia satisfaz o predicado inicial e que cada par de cópias consecutivas satisfazem o predicado de transição.

A seguinte função cria a $i$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [6]:
def declare(i): 
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = Int('x'+str(i))
    return state

### Exercício 1

Defina a função `init` que, dado um possível estado do programa (um dicionário de variáveis), devolva um predicado Z3 que testa se esse estado é um possível estado inicial do programa.

In [7]:
def init(state):
    # completar
    return And(state['pc'] == 0, state['x']>=3)
    

### Exercício 2

Defina a função `trans` que, dados dois possíveis estados do programa, devolva um predicado Z3 que testa se é possível transitar do primeiro para o segundo.

In [18]:
def trans(curr,prox):
    # completar
    t1 = And (curr['pc'] == 0, curr['x']>0,prox['pc']==1,prox['x']==curr['x'])
    t2 = And (curr['pc'] == 0, curr['x']>=0, prox['pc'] == 2, prox['x'] == curr['x'])
    t3 = And (curr['pc'] == 1, prox['pc']==0, prox['x'] == curr['x']-1)
    t4 = And (curr['pc'] == 2, prox['pc']==2, prox['x'] == curr['x'])
    return Or(t1,t2,t3,t4)
    
    
    

### Exercício 3

Complete a função de ordem superior `gera_traco` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, e um número positivo `k`, use o Z3 para gerar um possível traço de execução do programa de tamanho `k`. Para cada estado do traço deverá imprimir o respectivo valor das variáveis.

In [22]:
def gera_traco(declare,init,trans,k):
    s = Solver()
    # completar
    traco = {}
    for i in range(k): 
        traco[i] = declare(i)
    s.add(init(traco[0]))
    
    for i in range(k-1): 
        s.add(trans(traco[i],traco[i+1]))
    
    status = s.check() 
    if status == sat: 
        m = s.model()
        for i in range(k): 
            print(i)
            for v in traco[i]: 
                print (v,"=",m[traco[i][v]].as_long())
    elif status == unsat: 
        print("Nao ha execucoes possiveis")
    else: 
        print("No idea :-(")
        
gera_traco(declare,init,trans,10)

0
pc = 0
x = 3
1
pc = 1
x = 3
2
pc = 0
x = 2
3
pc = 1
x = 2
4
pc = 0
x = 1
5
pc = 2
x = 1
6
pc = 2
x = 1
7
pc = 2
x = 1
8
pc = 2
x = 1
9
pc = 2
x = 1


## Lógica temporal linear (LTL)

Sobre este FOTS podemos querer verificar várias propriedades temporais, como por exemplo:
1. $x$ é sempre positivo
2. $x$ é sempre diferente de 1
3. $x$ chega inevitavelmente a 0
4. $x$ chega inevitavelmente a um valor negativo
5. o programa termina

Estas propriedades são de natureza muito diferente. As duas primeiras são *propriedades de segurança* (*safety*) que, em geral, garantem que "nada de mau irá acontecer". Neste caso em particular são invariantes, ou seja, propriedades que devem ser verdade em todos os estados da execução do programa. As três últimas são *propriedades de animação* (*liveness*) que, em geral, garantem que "algo de bom irá acontecer".

A lógica LTL introduz *operadores temporais* que nos permitem escrever estas propriedades formalmente. Os operadores mais conhecidos são o $G$, que informalmente significa "*é sempre verdade que*", e o $F$, que informalmente significa "*é inevitável que*". Com estes operadores, as propriedades acima podem ser especificadas formalmente do seguinte modo
1. $G\ (x \ge 0)$
2. $G\ (x \neq 1)$
3. $F\ (x = 0)$
4. $F\ (x < 0)$
5. $F\ (pc = 2)$

## *Bounded Model Checking* (BMC) para LTL

Como é óbvio, nem todas estas propriedades são válidas. Em particular a 2ª e a 4ª não o são. O objectivo da verificação é precisamente determinar se uma propriedade temporal é válida num FOTS ou não. Este procedimento designa-se *model checking* e, quando uma propriedade não é válida, produz um contra-exemplo (um traço do FOTS correspondente a uma execução do programa onde a propriedade falha). Nesta aula vamos estudar uma técnica particular de *model checking* designada *bounded model checking*, onde o objectivo é determinar se uma propriedade temporal é válida nos primeiros $K$ estados da execução do FOTS.

A ideia passa por usar um SMT solver (neste caso o Z3) para tentar descobrir um contra-exemplo para uma dada propriedade. Para simplificar vamos abordar apenas dois casos: verificação de invariantes da forma $G\ \phi$ e propriedades de animação simples da forma $F\ \phi$, em que $\phi$ é uma fórmula sem operadores temporais.

### BMC de invariantes

Para fazer BMC de um invariante, por exemplo $G\ (x \ge 0)$ basta-nos usar o Z3 para encontrar um contra-exemplo com no máximo $K$ estados onde a propriedade $(x \ge 0)$ seja inválida nalgum estado. Para tal podemos implementar um procedimento iterativo que tenta encontrar esse contra-exemplo com tamanhos crescentes, começando com tamanho 1 até ao tamanho máximo $K$. Para cada tamanho $0 < k \le K$ basta tentar encontrar um possível traço onde o invariante a verificar seja inválido no último estado. Para tal podemos usar um código muito semelhante ao da função `gera_traco`. O procedimento é interrompido mal um contra-exemplo seja encontrado, sendo garantido que esse é um contra-exemplo mínimo para essa propriedade.

### Exercício 4

Complete a definição da função de ordem superior `bmc_always` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, um invariante a verificar, e um número positivo `K`, use o Z3 para verificar se esse invariante é sempre válido nos primeiros `K-1` passos de execução do programa, ou devolva um contra-exemplo mínimo caso não seja.

In [27]:
def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        s = Solver()
        # completar
        traco = {}
        for i in range(k): 
            traco[i] = declare(i)
        s.add(init(traco[0]))

        for i in range(k-1): 
            s.add(trans(traco[i],traco[i+1]))
        s.add(Not(inv(traco[k-1]))) #Procura onde falha
        status = s.check() 
        if status == sat: 
            m = s.model()
            for i in range(k): 
                print(i)
                for v in traco[i]: 
                    print (v,"=",m[traco[i][v]].as_long())
            return 
    print("A propriedade pode ser verdade")

        
def positive(state):
    return (state['x'] != 0) #Propriedade que queremos verificar 

bmc_always(declare,init,trans,positive,20) #retorna o contra exemplo mais pequeno

0
pc = 0
x = 3
1
pc = 1
x = 3
2
pc = 0
x = 2
3
pc = 1
x = 2
4
pc = 0
x = 1
5
pc = 1
x = 1
6
pc = 0
x = 0


### Exercício 5

Use o procedimento `bmc_always` para encontrar um contra-exemplo para a segunda propriedade especificada acima.

In [ ]:
# completar


### BMC de propriedades de animação

Para fazer BMC de propriedades de animação da forma $F\ \phi$ o prodimento é ligeiramente mais complicado. Neste caso não basta encontrar um traço aberto com no máximo $K$ estados onde $\phi$ nunca seja válida, pois tal contra-exemplo não seria convincente: nada garante que $\phi$ não pudesse ser válida num ponto mais tarde da execução. Neste caso, o contra-exemplo teria que ser uma execução completa do programa que demonstre inequivocamente que a propriedade não é válida. É possível encontrar uma execução completa do programa com no máximo $K$ estados se a mesma tiver um *loop* no último estado, mais concretamente, uma transição para um dos estados precedentes. 

### Exercício 6

Complete a definição da função de ordem superior `bmc_eventually` que, dada uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, uma propriedade cuja inevitabilidade se pretende verificar, e um número positivo `K`, use o Z3 para encontrar um contra-exemplo para essa propriedade considerando apenas os primeiros `K` estados de execução do programa. Note que neste caso um contra-exemplo tem que ser necessariamente um *loop* (no sentido acima referido) onde a propriedade desejada nunca seja válida.

In [35]:
 def bmc_eventually(declare,init,trans,prop,K):
    for k in range(1,K+1):
        s = Solver()
        # completar
        traco = {}
        for i in range(k):
          traco[i] = declare(i)
        s.add(init(traco[0]))
        for i in range(k-1):
          s.add(trans(traco[i],traco[i+1]))

        for i in range(k):
          s.add(Not(prop(traco[i])))
        s.add(Or([trans(traco[k-1], traco[i]) for i in range(k)]))

        status = s.check()
        if status == sat:
          m = s.model()
          for i in range(k):
            print(i)
            for v in traco[i]:
              print(v,"=",m[traco[i][v]])
          return
    print("A prop pode ser True")

def zero(state):
    return (state['x'] == 0)

bmc_eventually(declare,init,trans,zero,20)

def terminates(state):
    return (state['pc'] == 2)

bmc_eventually(declare,init,trans,terminates,20)

0
pc = 0
x = 3
1
pc = 2
x = 3
A prop pode ser True


### Exercício 7

Modifique a função `bmc_eventually` para não só imprimir os estados do contra-exemplo, mas também o estado onde começa o *loop*. Sugere-se a utilização da função `eval` do Z3 para detectar esse estado. Utilize esta função modificada para encontrar um contra exemplo para a quarta propriedade acima referida.

In [ ]:
def bmc_eventually(declare,init,trans,prop,K):
    for k in range(1,K+1):
        s = Solver()
        # completar
        
    
def negative(state):
    return (state['x'] < 0)

bmc_eventually(declare,init,trans,negative,10)